In [1]:
!pip install -q torch transformers datasets rouge-score tqdm pandas

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.5 

In [2]:
from huggingface_hub import login
login("1234567890")

In [4]:
"""
Step 1: Simple LLM Setup and Baseline Evaluation
================================================
Using TinyLlama-1.1B for maximum compatibility and speed
"""

import torch
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from rouge_score import rouge_scorer
import json
from tqdm import tqdm

class SimpleLLMBaseline:
    def __init__(self):
        # Use TinyLlama (1.1B params) - very stable and fast
        self.model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
        
        self.tokenizer = None
        self.model = None
        self.test_data = None
    
    def setup_model(self):
        """Load the model and tokenizer"""
        print("Loading TinyLlama model...")
        
        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # Load model
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        
        print("Model loaded successfully!")
        print(f"Model parameters: {self.model.num_parameters() / 1e9:.2f}B")
    
    def load_squad_data(self, num_examples=100):
        """Load a small subset of SQuAD dataset"""
        print(f"Loading SQuAD dataset ({num_examples} examples)...")
        
        # Load SQuAD validation set
        dataset = load_dataset("squad", split="validation")
        
        # Take a small subset and format it properly
        test_examples = []
        
        for i, example in enumerate(dataset):
            if i >= num_examples:
                break
                
            # SQuAD format: context, question, answers
            test_examples.append({
                'context': example['context'],
                'question': example['question'], 
                'answer': example['answers']['text'][0],  # First answer
                'id': example['id']
            })
        
        self.test_data = test_examples
        print(f"Loaded {len(self.test_data)} test examples")
        
        # Show a sample
        print("\nSample example:")
        sample = self.test_data[0]
        print(f"Question: {sample['question']}")
        print(f"Context: {sample['context'][:200]}...")
        print(f"Answer: {sample['answer']}")
        
        return self.test_data
    
    def create_prompt(self, context, question):
        """Create a simple prompt for TinyLlama"""
        prompt = f"""<|system|>
You are a helpful assistant. Answer the question based on the context provided. Give a brief, direct answer.
</s>
<|user|>
Context: {context}

Question: {question}

Answer:
</s>
<|assistant|>
"""
        return prompt
    
    def generate_answer(self, context, question, max_new_tokens=30):
        """Generate an answer for a given context and question"""
        prompt = self.create_prompt(context, question)
        
        # Tokenize input - keep it shorter for TinyLlama
        inputs = self.tokenizer(
            prompt, 
            return_tensors="pt", 
            truncation=True, 
            max_length=1500  # Shorter for TinyLlama
        ).to(self.device)
        
        # Generate response with simpler parameters
        with torch.no_grad():
            outputs = self.model.generate(
                inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=max_new_tokens,
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
                # Remove problematic parameters
                use_cache=True
            )
        
        # Decode the response
        full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extract only the assistant's response
        if "<|assistant|>" in full_response:
            answer = full_response.split("<|assistant|>")[-1].strip()
        else:
            # Fallback: extract everything after the prompt
            answer = full_response[len(self.tokenizer.decode(inputs.input_ids[0], skip_special_tokens=True)):].strip()
        
        # Clean up the answer
        answer = answer.split('\n')[0].strip()  # Take first line only
        return answer
    
    def evaluate_baseline(self, num_examples=50):
        """Evaluate baseline model performance"""
        print(f"Evaluating baseline on {num_examples} examples...")
        
        rouge_scorer_obj = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        
        results = []
        rouge_scores = []
        
        for i, example in enumerate(tqdm(self.test_data[:num_examples])):
            try:
                # Generate prediction
                prediction = self.generate_answer(
                    example['context'], 
                    example['question']
                )
                
                # Clean prediction
                if not prediction.strip():
                    prediction = "No answer"
                
                # Calculate ROUGE-L score
                score = rouge_scorer_obj.score(
                    example['answer'], 
                    prediction
                )
                rouge_l_score = score['rougeL'].fmeasure
                
                results.append({
                    'id': example['id'],
                    'question': example['question'],
                    'true_answer': example['answer'],
                    'predicted_answer': prediction,
                    'rouge_l': rouge_l_score
                })
                
                rouge_scores.append(rouge_l_score)
                
            except Exception as e:
                print(f"Error processing example {i}: {e}")
                continue
        
        # Calculate average score
        if rouge_scores:
            avg_rouge = sum(rouge_scores) / len(rouge_scores)
        else:
            avg_rouge = 0.0
        
        print(f"\n=== BASELINE RESULTS ===")
        print(f"Average ROUGE-L Score: {avg_rouge:.4f}")
        print(f"Number of examples processed: {len(rouge_scores)}")
        
        # Show some examples
        print(f"\n=== SAMPLE PREDICTIONS ===")
        for i in range(min(5, len(results))):
            result = results[i]
            print(f"\nExample {i+1}:")
            print(f"Question: {result['question']}")
            print(f"True Answer: {result['true_answer']}")
            print(f"Predicted: {result['predicted_answer']}")
            print(f"ROUGE-L: {result['rouge_l']:.4f}")
            print("-" * 50)
        
        return {
            'average_rouge_l': avg_rouge,
            'all_scores': rouge_scores,
            'detailed_results': results
        }
    
    def run_baseline_test(self, num_data_examples=100, num_eval_examples=50):
        """Run the complete baseline test"""
        print("🚀 Starting Simple Baseline Test with TinyLlama")
        print("=" * 50)
        
        # Setup
        self.setup_model()
        self.load_squad_data(num_data_examples)
        
        # Evaluate
        results = self.evaluate_baseline(num_eval_examples)
        
        print(f"\n✅ Baseline test completed!")
        print(f"Final ROUGE-L Score: {results['average_rouge_l']:.4f}")
        
        # Interpretation
        score = results['average_rouge_l']
        if score > 0.30:
            print("🎉 Great! This is a good baseline to build upon.")
        elif score > 0.20:
            print("✅ Decent baseline. We can improve with RAG/fine-tuning.")
        elif score > 0.10:
            print("⚠️ Low but workable. May need prompt tuning.")
        else:
            print("❌ Very low score. Need to debug.")
        
        return results

# Test the baseline
if __name__ == "__main__":
    baseline = SimpleLLMBaseline()
    results = baseline.run_baseline_test(num_data_examples=200, num_eval_examples=100)

Using device: cuda
🚀 Starting Simple Baseline Test with TinyLlama
Loading TinyLlama model...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model loaded successfully!
Model parameters: 1.10B
Loading SQuAD dataset (200 examples)...
Loaded 200 test examples

Sample example:
Question: Which NFL team represented the AFC at Super Bowl 50?
Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated...
Answer: Denver Broncos
Evaluating baseline on 100 examples...


100%|██████████| 100/100 [01:28<00:00,  1.12it/s]


=== BASELINE RESULTS ===
Average ROUGE-L Score: 0.1909
Number of examples processed: 100

=== SAMPLE PREDICTIONS ===

Example 1:
Question: Which NFL team represented the AFC at Super Bowl 50?
True Answer: Denver Broncos
Predicted: The NFL team representing the American Football Conference (AFC) at Super Bowl 50 was the Denver Broncos.
ROUGE-L: 0.2105
--------------------------------------------------

Example 2:
Question: Which NFL team represented the NFC at Super Bowl 50?
True Answer: Carolina Panthers
Predicted: The NFL team represented the NFC at Super Bowl 50 was the Carolina Panthers.
ROUGE-L: 0.2500
--------------------------------------------------

Example 3:
Question: Where did Super Bowl 50 take place?
True Answer: Santa Clara, California
Predicted: Super Bowl 50 took place at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California, which is the host city
ROUGE-L: 0.2222
--------------------------------------------------

Example 4:
Question: Which NFL team 

In [4]:
import torch
print(torch.version.cuda)

12.4


In [5]:
# For CUDA 12.x, this should work
!pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 55.0 MB/s eta 0:00:00:00:0100:01


In [6]:
import faiss
print(faiss.__version__)

1.12.0


In [7]:
"""
Step 2: Simple RAG Implementation
=================================
Adding retrieval-augmented generation to improve the baseline
"""

import torch
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from rouge_score import rouge_scorer
import json
from tqdm import tqdm

class SimpleRAGPipeline:
    def __init__(self):
        
        # Same model as baseline for fair comparison
        self.model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
        
        # Models
        self.tokenizer = None
        self.model = None
        self.embedding_model = None
        
        # RAG components
        self.faiss_index = None
        self.knowledge_passages = []
        
        # Data
        self.test_data = None
        self.knowledge_data = None
    
    def setup_models(self):
        """Load all required models"""
        print("Loading models...")
        
        # Load language model (same as baseline)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        
        # Load embedding model for retrieval
        print("Loading embedding model...")
        self.embedding_model = SentenceTransformer(
                                    "Qwen/Qwen3-Embedding-4B",
                                    trust_remote_code=True,
                                    model_kwargs={
                                     #   "device_map": "auto",
                                        "torch_dtype": torch.float16 # half the memory and very less impact on model's performance
                                    }
                                )
        
        print("All models loaded successfully!")
    
    def load_and_split_data(self, total_examples=300):
        """Load SQuAD and split into knowledge base and test set"""
        print(f"Loading and splitting SQuAD data...")
        
        # Load SQuAD validation set
        dataset = load_dataset("squad", split="validation")
        
        # Take subset
        subset = dataset.select(range(min(total_examples, len(dataset))))
        
        # Split: 60% knowledge base, 40% test
        split_point = int(0.6 * len(subset))
        
        # Knowledge base (for RAG retrieval)
        kb_data = []
        for example in subset.select(range(split_point)):
            kb_data.append({
                'context': example['context'],
                'question': example['question'],
                'answer': example['answers']['text'][0],
                'id': example['id']
            })
        
        # Test data (for evaluation)
        test_data = []
        for example in subset.select(range(split_point, len(subset))):
            test_data.append({
                'context': example['context'],
                'question': example['question'],
                'answer': example['answers']['text'][0],
                'id': example['id']
            })
        
        self.knowledge_data = kb_data
        self.test_data = test_data
        
        print(f"Data split - Knowledge base: {len(kb_data)}, Test: {len(test_data)}")
        
        # Show sample
        print(f"\nSample test question: {test_data[0]['question']}")
        print(f"Sample knowledge context: {kb_data[0]['context'][:100]}...")
        
        return kb_data, test_data
    
    def build_knowledge_index(self):
        """Build FAISS index from knowledge base contexts"""
        print("Building FAISS index...")
        
        # Extract all contexts from knowledge base
        contexts = [item['context'] for item in self.knowledge_data]
        self.knowledge_passages = contexts
        
        # Generate embeddings for all contexts
        print(f"Length of contexts = {len(contexts)}")
        print("Generating embeddings...")
        emb_list = []
        for single_context in contexts:
            curr_emb = self.embedding_model.encode(
                [single_context],
                convert_to_numpy=True,
                normalize_embeddings=True,
                show_progress_bar=False   # outer loop can print progress instead
            )
            emb_list.append(curr_emb[0])
        
        embeddings = np.vstack(emb_list).astype("float32", copy=False)
        
        # Create FAISS index
        dimension = embeddings.shape[1]
        self.faiss_index = faiss.IndexFlatIP(dimension)  # Inner product for similarity
        self.faiss_index.add(embeddings)
        
        print(f"FAISS index built with {len(contexts)} passages")
        print(f"Embedding dimension: {dimension}")
    
    def retrieve_relevant_contexts(self, question, top_k=2):
        """Retrieve top-k most relevant contexts for a question"""
        # Encode the question
        question_embedding = self.embedding_model.encode([question],convert_to_numpy=True,normalize_embeddings=True,show_progress_bar=False)
        
        # Search for similar contexts
        scores, indices = self.faiss_index.search(
            question_embedding, 
            top_k
        )
        
        # Get the relevant contexts
        relevant_contexts = []
        for idx in indices[0]:
            relevant_contexts.append(self.knowledge_passages[idx])
        
        return relevant_contexts, scores[0]
    
    def create_rag_prompt(self, question, retrieved_contexts):
        """Create RAG prompt with retrieved contexts"""
        # Combine retrieved contexts
        combined_context = "\n\n".join(retrieved_contexts)
        
        # Create prompt
        prompt = f"""<|system|>
You are a helpful assistant. Use the provided contexts to answer the question. Give a brief, direct answer based on the information given.
</s>
<|user|>
Contexts:
{combined_context}

Question: {question}

Answer:
</s>
<|assistant|>
"""
        return prompt
    
    def generate_rag_answer(self, question, max_new_tokens=30):
        """Generate answer using RAG approach"""
        # Step 1: Retrieve relevant contexts
        retrieved_contexts, scores = self.retrieve_relevant_contexts(question, top_k=2)
        
        # Step 2: Create RAG prompt
        prompt = self.create_rag_prompt(question, retrieved_contexts)
        
        # Step 3: Generate answer
        inputs = self.tokenizer(
            prompt, 
            return_tensors="pt", 
            truncation=True, 
            max_length=1800  # Longer to accommodate retrieved contexts
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model.generate(
                inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=max_new_tokens,
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
                use_cache=True
            )
        
        # Decode response
        full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(full_response)
        
        # Extract assistant's response
        if "<|assistant|>" in full_response:
            answer = full_response.split("<|assistant|>")[-1].strip()
        else:
            answer = full_response[len(self.tokenizer.decode(inputs.input_ids[0], skip_special_tokens=True)):].strip()
        
        # Clean answer
        answer = answer.split('\n')[0].strip()
        
        return answer, retrieved_contexts, scores
    
    def evaluate_rag(self, num_examples=50):
        """Evaluate RAG performance"""
        print(f"Evaluating RAG on {num_examples} examples...")
        
        rouge_scorer_obj = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        
        results = []
        rouge_scores = []
        
        for i, example in enumerate(tqdm(self.test_data[:num_examples])):
            try:
                # Generate RAG prediction
                prediction, retrieved_contexts, retrieval_scores = self.generate_rag_answer(
                    example['question']
                )
                
                if not prediction.strip():
                    prediction = "No answer"
                
                # Calculate ROUGE-L score
                score = rouge_scorer_obj.score(
                    example['answer'], 
                    prediction
                )
                rouge_l_score = score['rougeL'].fmeasure
                
                results.append({
                    'id': example['id'],
                    'question': example['question'],
                    'true_answer': example['answer'],
                    'predicted_answer': prediction,
                    'rouge_l': rouge_l_score,
                    'retrieved_contexts': retrieved_contexts,
                    'retrieval_scores': retrieval_scores.tolist()
                })
                
                rouge_scores.append(rouge_l_score)
                
            except Exception as e:
                print(f"Error processing example {i}: {e}")
                continue
        
        # Calculate average score
        avg_rouge = sum(rouge_scores) / len(rouge_scores) if rouge_scores else 0.0
        
        print(f"\n=== RAG RESULTS ===")
        print(f"Average ROUGE-L Score: {avg_rouge:.4f}")
        print(f"Number of examples processed: {len(rouge_scores)}")
        
        # Show sample predictions with retrieval info
        print(f"\n=== SAMPLE RAG PREDICTIONS ===")
        for i in range(min(3, len(results))):
            result = results[i]
            print(f"\nExample {i+1}:")
            print(f"Question: {result['question']}")
            print(f"True Answer: {result['true_answer']}")
            print(f"RAG Answer: {result['predicted_answer']}")
            print(f"ROUGE-L: {result['rouge_l']:.4f}")
            print(f"Retrieved contexts preview:")
            for j, ctx in enumerate(result['retrieved_contexts']):
                print(f"  Context {j+1}: {ctx[:100]}...")
            print("-" * 50)
        
        return {
            'average_rouge_l': avg_rouge,
            'all_scores': rouge_scores,
            'detailed_results': results
        }
    
    def run_rag_pipeline(self, total_examples=300, eval_examples=50):
        """Run the complete RAG pipeline"""
        print("🚀 Starting RAG Pipeline")
        print("=" * 40)
        
        # Setup
        self.setup_models()
        self.load_and_split_data(total_examples)
        self.build_knowledge_index()
        
        # Evaluate RAG
        results = self.evaluate_rag(eval_examples)
        
        print(f"\n✅ RAG pipeline completed!")
        print(f"RAG ROUGE-L Score: {results['average_rouge_l']:.4f}")
        
        # Compare with baseline
        baseline_score = 0.1909  # From your previous run
        improvement = results['average_rouge_l'] - baseline_score
        
        print(f"\n=== COMPARISON ===")
        print(f"Baseline: {baseline_score:.4f}")
        print(f"RAG:      {results['average_rouge_l']:.4f}")
        print(f"Improvement: {improvement:+.4f}")
        
        if improvement > 0.02:
            print("🎉 RAG shows significant improvement!")
        elif improvement > 0:
            print("✅ RAG shows some improvement.")
        else:
            print("⚠️ RAG didn't improve. May need tuning.")
        
        return results

# Test RAG implementation
if __name__ == "__main__":
    rag_pipeline = SimpleRAGPipeline()
    results = rag_pipeline.run_rag_pipeline(total_examples=300, eval_examples=100)

2025-08-16 11:49:18.441944: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755344958.647259      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755344958.700426      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda
🚀 Starting RAG Pipeline
Loading models...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

<|system|>
You are a helpful assistant. Use the provided contexts to answer the question. Give a brief, direct answer based on the information given.
 
<|user|>
Contexts:
The Broncos took an early lead in Super Bowl 50 and never trailed. Newton was limited by Denver's defense, which sacked him seven times and forced him into three turnovers, including a fumble which they recovered for a touchdown. Denver linebacker Von Miller was named Super Bowl MVP, recording five solo tackles, 2½ sacks, and two forced fumbles.

The Broncos took an early lead in Super Bowl 50 and never trailed. Newton was limited by Denver's defense, which sacked him seven times and forced him into three turnovers, including a fumble which they recovered for a touchdown. Denver linebacker Von Miller was named Super Bowl MVP, recording five solo tackles, 2½ sacks, and two forced fumbles.

Question: What was the number of 17 interceptions that Peyton Manning had by end of the year?

Answer:
 
<|assistant|>
The number o

 99%|█████████▉| 99/100 [01:38<00:00,  1.03it/s]

<|system|>
You are a helpful assistant. Use the provided contexts to answer the question. Give a brief, direct answer based on the information given.
 
<|user|>
Contexts:
The Broncos took an early lead in Super Bowl 50 and never trailed. Newton was limited by Denver's defense, which sacked him seven times and forced him into three turnovers, including a fumble which they recovered for a touchdown. Denver linebacker Von Miller was named Super Bowl MVP, recording five solo tackles, 2½ sacks, and two forced fumbles.

The Broncos took an early lead in Super Bowl 50 and never trailed. Newton was limited by Denver's defense, which sacked him seven times and forced him into three turnovers, including a fumble which they recovered for a touchdown. Denver linebacker Von Miller was named Super Bowl MVP, recording five solo tackles, 2½ sacks, and two forced fumbles.

Question: Who had the most receptions out of all players for the year?

Answer:
 
<|assistant|>
Cody Latimer had the most reception

100%|██████████| 100/100 [01:39<00:00,  1.01it/s]

<|system|>
You are a helpful assistant. Use the provided contexts to answer the question. Give a brief, direct answer based on the information given.
 
<|user|>
Contexts:
The Broncos took an early lead in Super Bowl 50 and never trailed. Newton was limited by Denver's defense, which sacked him seven times and forced him into three turnovers, including a fumble which they recovered for a touchdown. Denver linebacker Von Miller was named Super Bowl MVP, recording five solo tackles, 2½ sacks, and two forced fumbles.

The Broncos took an early lead in Super Bowl 50 and never trailed. Newton was limited by Denver's defense, which sacked him seven times and forced him into three turnovers, including a fumble which they recovered for a touchdown. Denver linebacker Von Miller was named Super Bowl MVP, recording five solo tackles, 2½ sacks, and two forced fumbles.

Question: What position does Demaryius Thomas play?

Answer:
 
<|assistant|>
Demaryius Thomas is a wide receiver for the Broncos in

In [7]:
"""
Ultra Simple LoRA Fine-tuning - Avoiding all batching/tokenization issues
=========================================================================
This version pre-processes everything and uses the simplest possible approach
"""

import torch
from torch.optim import AdamW
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
from rouge_score import rouge_scorer
from tqdm import tqdm
import json

class UltraSimpleLoRAFinetune:
    def __init__(self):
        self.model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
        
        # Models
        self.tokenizer = None
        self.base_model = None
        self.finetuned_model = None
        
        # Pre-processed data
        self.train_inputs = []
        self.test_data = []
    
    def setup_model(self):
        """Load model and tokenizer"""
        print("Loading model and tokenizer...")
        
        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # Load base model
        self.base_model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        
        print("Model loaded successfully!")
    
    def prepare_data(self):
        """Prepare and pre-tokenize all data"""
        print("Loading and preparing data...")
        
        # Load SQuAD dataset
        dataset = load_dataset("squad", split="train")
        
        # Take smaller subsets to avoid issues
        train_examples = dataset.select(range(500))  # Only 500 for training
        test_examples = dataset.select(range(500, 700))  # 200 for testing
        
        print("Pre-tokenizing training data...")
        
        # Pre-tokenize all training examples
        self.train_inputs = []
        max_length = 400  # Keep it short
        
        for i, example in enumerate(tqdm(train_examples)):
            # Create training text
            text = f"""<|system|>
Answer the question based on the context.
</s>
<|user|>
Context: {example['context'][:500]}

Question: {example['question']}
</s>
<|assistant|>
{example['answers']['text'][0]}</s>"""
            
            # Tokenize
            try:
                encoded = self.tokenizer(
                    text,
                    truncation=True,
                    padding=False,
                    max_length=max_length,
                    return_tensors="pt"
                )
                
                # Only keep if reasonable length
                if len(encoded.input_ids[0]) > 50 and len(encoded.input_ids[0]) <= max_length:
                    self.train_inputs.append({
                        'input_ids': encoded.input_ids[0],
                        'attention_mask': encoded.attention_mask[0]
                    })
                    
            except Exception as e:
                print(f"Skipping example {i}: {e}")
                continue
        
        # Prepare test data (simple format)
        self.test_data = []
        for example in test_examples:
            self.test_data.append({
                'context': example['context'],
                'question': example['question'],
                'answer': example['answers']['text'][0]
            })
        
        print(f"Training examples prepared: {len(self.train_inputs)}")
        print(f"Test examples: {len(self.test_data)}")
    
    def setup_lora(self):
        """Setup LoRA configuration"""
        print("Setting up LoRA...")
        
        # Simple LoRA config
        lora_config = LoraConfig(
            task_type=TaskType.CAUSAL_LM,
            inference_mode=False,
            r=8,  # Smaller rank
            lora_alpha=16,
            lora_dropout=0.1,
            target_modules=["q_proj", "v_proj"],  # Only target these modules
            bias="none"
        )
        
        # Apply LoRA
        self.finetuned_model = get_peft_model(self.base_model, lora_config)
        
        # Print trainable parameters
        trainable_params = sum(p.numel() for p in self.finetuned_model.parameters() if p.requires_grad)
        total_params = sum(p.numel() for p in self.finetuned_model.parameters())
        
        print(f"Trainable parameters: {trainable_params:,}")
        print(f"Total parameters: {total_params:,}")
        print(f"Trainable %: {100 * trainable_params / total_params:.2f}%")
    
    def simple_train(self):
        """Ultra simple training loop"""
        print("Starting simple training loop...")
        
        # Setup optimizer
        optimizer = AdamW(self.finetuned_model.parameters(), lr=1e-4)
        
        # Training parameters
        num_epochs = 2
        batch_size = 1  # Process one at a time to avoid batching issues
        
        self.finetuned_model.train()
        
        for epoch in range(num_epochs):
            print(f"\nEpoch {epoch + 1}/{num_epochs}")
            
            total_loss = 0
            num_batches = 0
            
            # Shuffle training data
            import random
            random.shuffle(self.train_inputs)
            
            # Process examples one by one
            for i, batch in enumerate(tqdm(self.train_inputs)):
                try:
                    # Move to device
                    input_ids = batch['input_ids'].unsqueeze(0).to(self.device)
                    attention_mask = batch['attention_mask'].unsqueeze(0).to(self.device)
                    
                    # Forward pass
                    outputs = self.finetuned_model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=input_ids  # For causal LM, labels = input_ids
                    )
                    
                    loss = outputs.loss
                    
                    # Backward pass
                    loss.backward()
                    
                    # Update every 4 steps (gradient accumulation)
                    if (i + 1) % 4 == 0:
                        optimizer.step()
                        optimizer.zero_grad()
                    
                    total_loss += loss.item()
                    num_batches += 1
                    
                    # Log every 50 steps
                    if (i + 1) % 50 == 0:
                        avg_loss = total_loss / num_batches
                        print(f"Step {i + 1}, Average Loss: {avg_loss:.4f}")
                
                except Exception as e:
                    print(f"Error processing batch {i}: {e}")
                    continue
            
            avg_loss = total_loss / num_batches if num_batches > 0 else 0
            print(f"Epoch {epoch + 1} completed. Average Loss: {avg_loss:.4f}")
        
        print("Training completed!")
    
    def generate_answer(self, context, question, max_new_tokens=30):
        """Generate answer using fine-tuned model"""
        prompt = f"""<|system|>
Answer the question based on the context.
</s>
<|user|>
Context: {context[:500]}

Question: {question}
</s>
<|assistant|>
"""
        
        # Tokenize
        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=400
        ).to(self.device)
        
        # Generate
        self.finetuned_model.eval()
        with torch.no_grad():
            outputs = self.finetuned_model.generate(
                inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=max_new_tokens,
                temperature=0.3,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id
            )
        
        # Decode
        full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extract answer
        if "<|assistant|>" in full_response:
            answer = full_response.split("<|assistant|>")[-1].strip()
        else:
            answer = full_response[len(prompt):].strip()
        
        # Clean answer
        answer = answer.split('\n')[0].strip()
        return answer
    
    def evaluate_model(self, num_examples=100):
        """Evaluate the fine-tuned model"""
        print(f"Evaluating fine-tuned model on {num_examples} examples...")
        
        rouge_scorer_obj = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        
        results = []
        rouge_scores = []
        
        for i, example in enumerate(tqdm(self.test_data[:num_examples])):
            try:
                # Generate prediction
                prediction = self.generate_answer(
                    example['context'],
                    example['question']
                )
                
                if not prediction.strip():
                    prediction = "No answer"
                
                # Calculate ROUGE-L
                score = rouge_scorer_obj.score(
                    example['answer'],
                    prediction
                )
                rouge_l_score = score['rougeL'].fmeasure
                
                results.append({
                    'question': example['question'],
                    'true_answer': example['answer'],
                    'predicted_answer': prediction,
                    'rouge_l': rouge_l_score
                })
                
                rouge_scores.append(rouge_l_score)
                
            except Exception as e:
                print(f"Error evaluating example {i}: {e}")
                continue
        
        # Calculate average
        avg_rouge = sum(rouge_scores) / len(rouge_scores) if rouge_scores else 0.0
        
        print(f"\n=== FINE-TUNED RESULTS ===")
        print(f"Average ROUGE-L Score: {avg_rouge:.4f}")
        print(f"Examples processed: {len(rouge_scores)}")
        
        # Show samples
        print(f"\n=== SAMPLE PREDICTIONS ===")
        for i in range(min(3, len(results))):
            result = results[i]
            print(f"\nExample {i+1}:")
            print(f"Question: {result['question']}")
            print(f"True Answer: {result['true_answer']}")
            print(f"Fine-tuned Answer: {result['predicted_answer']}")
            print(f"ROUGE-L: {result['rouge_l']:.4f}")
            print("-" * 50)
        
        return avg_rouge
    
    def run_complete_pipeline(self):
        """Run the complete pipeline"""
        print("🚀 Starting Ultra Simple Fine-tuning Pipeline")
        print("=" * 60)
        
        # Setup everything
        self.setup_model()
        self.prepare_data()
        self.setup_lora()
        
        # Train
        self.simple_train()
        
        # Evaluate
        score = self.evaluate_model()
        
        print(f"\n✅ Pipeline completed!")
        print(f"Fine-tuned ROUGE-L Score: {score:.4f}")
        
        # Comparison
        baseline_score = 0.1909
        improvement = score - baseline_score
        
        print(f"\n=== FINAL COMPARISON ===")
        print(f"Baseline:   {baseline_score:.4f}")
        print(f"Fine-tuned: {score:.4f}")
        print(f"Improvement: {improvement:+.4f}")
        
        if improvement > 0.05:
            print("🎉 Significant improvement!")
        elif improvement > 0:
            print("✅ Some improvement achieved!")
        else:
            print("⚠️ No improvement, but training completed successfully!")
        
        return score

# Run the ultra simple pipeline
if __name__ == "__main__":
    finetuner = UltraSimpleLoRAFinetune()
    results = finetuner.run_complete_pipeline()

Using device: cuda
🚀 Starting Ultra Simple Fine-tuning Pipeline
Loading model and tokenizer...
Model loaded successfully!
Loading and preparing data...
Pre-tokenizing training data...


100%|██████████| 500/500 [00:00<00:00, 1374.05it/s]


Training examples prepared: 500
Test examples: 200
Setting up LoRA...
Trainable parameters: 1,126,400
Total parameters: 1,101,174,784
Trainable %: 0.10%
Starting simple training loop...

Epoch 1/2


 10%|█         | 51/500 [00:07<00:50,  8.89it/s]

Step 50, Average Loss: 2.0551


...

Step 450, Average Loss: 1.2770


100%|██████████| 500/500 [00:57<00:00,  8.63it/s]


Step 500, Average Loss: 1.2616
Epoch 2 completed. Average Loss: 1.2616
Training completed!
Evaluating fine-tuned model on 100 examples...


100%|██████████| 100/100 [00:27<00:00,  3.66it/s]


=== FINE-TUNED RESULTS ===
Average ROUGE-L Score: 0.6188
Examples processed: 100

=== SAMPLE PREDICTIONS ===

Example 1:
Question: Who did Beyonce donate the money to earned from her shows?
True Answer: Clinton Bush Haiti Fund
Fine-tuned Answer: the Clinton Bush Haiti Fund
ROUGE-L: 0.8889
--------------------------------------------------

Example 2:
Question: Beyonce became the first female artist to perform solo in 20 years at which stage?
True Answer: the 2011 Glastonbury Festival
Fine-tuned Answer: The Pyramid stage at the 2011 Glast
ROUGE-L: 0.3636
--------------------------------------------------

Example 3:
Question: Which organization did Beyonce's spokespeople confirm her donations to?
True Answer: The Huffington Post
Fine-tuned Answer: the Clinton Bush Haiti Fund
ROUGE-L: 0.2500
--------------------------------------------------

✅ Pipeline completed!
Fine-tuned ROUGE-L Score: 0.6188

=== FINAL COMPARISON ===
Baseline:   0.1909
Fine-tuned: 0.6188
Improvement: +0.4279
🎉 Sign